#Import libaries

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
import sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras import models, layers, regularizers
from sklearn.preprocessing import StandardScaler, OneHotEncoder

#Import Dataset from Kaggle

In [ ]:
from google.colab import files

#upload kaggle.json file
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"anniebradders","key":"d5cb7bdf93d0d48489d2e167b677f1ce"}'}

In [ ]:
! pip install -q kaggle
! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download -d charmainemicallef/f1dataset2

  0% 0.00/1.95M [00:00<?, ?B/s]
100% 1.95M/1.95M [00:00<00:00, 121MB/s]


In [ ]:
! unzip f1dataset2.zip -d f1dataset

Archive:  f1dataset2.zip
  inflating: f1dataset/f1dataset2.csv  


#Reading dataset

In [ ]:
! pip install joblib

In [ ]:
# read df
df = pd.read_csv(r'/content/f1dataset/f1dataset2.csv', encoding='utf-8')

# shuffle data
shuffled_data = df.sample(frac=1, random_state=42)  # Set random_state for reproducibility

In [ ]:
df

,race_id,lapno,position,driver_id,laptime,racetime,gap,interval,compound,tireage,...,availablecompounds,nolaps,nolapsplanned,tracklength,result_position,race_progress,remaining_pit_stops,relativecompound,fulfilled_second_compound,number_of_available_compounds
0,1,0,2,2,0.000,0.000,0.000,0.000,A3,2,...,"A2,A3,I,W",57,58,5303.0,2.0,0.000000,3,soft,True,2
1,1,1,2,2,103.549,103.549,1.511,1.511,A3,3,...,"A2,A3,I,W",57,58,5303.0,2.0,0.018480,3,soft,True,2
2,1,2,2,2,97.459,201.008,1.283,1.283,A3,4,...,"A2,A3,I,W",57,58,5303.0,2.0,0.035874,3,soft,True,2
3,1,3,2,2,96.321,297.329,1.839,1.839,A3,5,...,"A2,A3,I,W",57,58,5303.0,2.0,0.053064,3,soft,True,2
4,1,4,2,2,95.966,393.295,2.866,2.866,A3,6,...,"A2,A3,I,W",57,58,5303.0,2.0,0.070191,3,soft,True,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61325,121,51,5,43,102.376,5301.284,61.819,16.978,A4,38,...,"A4,A6,A7,I,W",55,55,5554.0,6.0,0.927622,3,medium,True,3
61326,121,52,5,43,102.462,5403.746,62.188,18.386,A4,39,...,"A4,A6,A7,I,W",55,55,5554.0,6.0,0.945551,3,medium,True,3
61327,121,53,6,43,104.788,5508.534,67.693,1.323,A4,40,...,"A4,A6,A7,I,W",55,55,5554.0,6.0,0.963886,3,medium,True,3
61328,121,54,6,43,103.028,5611.562,68.713,1.618,A4,41,...,"A4,A6,A7,I,W",55,55,5554.0,6.0,0.981914,3,medium,True,3


In [ ]:
#Check there are no empty/null values
df.isnull().sum()

race_id                          0
lapno                            0
position                         0
driver_id                        0
laptime                          0
racetime                         0
gap                              0
interval                         0
compound                         0
tireage                          0
pitintime                        0
pitstopduration                  0
nextcompound                     0
startlapprog_vsc                 0
endlapprog_vsc                   0
age_vsc                          0
startlapprog_sc                  0
endlapprog_sc                    0
age_sc                           0
id                               0
carno                            0
initials                         0
name                             0
date                             0
season                           0
location                         0
availablecompounds               0
nolaps                           0
nolapsplanned       

##Preproccessing Data

In [ ]:
from joblib import dump

# Generate a subset
subset_data = shuffled_data[['race_progress', 'remaining_pit_stops', 'relativecompound', 'location', 'fulfilled_second_compound', 'number_of_available_compounds']].sample(frac=0.1, random_state=42)

# Separate input features (X) and target variable (y)
X = shuffled_data[['race_progress', 'remaining_pit_stops', 'location', 'fulfilled_second_compound', 'number_of_available_compounds']]
y = shuffled_data['relativecompound']

# Separate categorical and numerical features
cat_features = ['remaining_pit_stops', 'location', 'fulfilled_second_compound', 'number_of_available_compounds']
num_features = ['race_progress']

In [ ]:
subset_data

,race_progress,remaining_pit_stops,relativecompound,location,fulfilled_second_compound,number_of_available_compounds
5169,0.056124,3,soft,Hockenheim,True,2
28468,0.924746,3,hard,Suzuka,True,2
27118,0.241679,3,soft,Singapore,True,2
50317,0.691500,3,medium,Melbourne,True,3
23419,0.985936,3,soft,Montreal,True,2
...,...,...,...,...,...,...
10912,0.019573,3,soft,YasMarina,True,2
14404,0.087492,3,soft,Montreal,True,2
22703,0.257823,3,medium,Catalunya,True,2
29006,0.420172,3,medium,MexicoCity,True,2


In [ ]:
from joblib import dump

# Perform preprocessing on numerical features
scaler = StandardScaler()
X[num_features] = scaler.fit_transform(X[num_features])

# Save the scaler
dump(scaler, 'scaler.joblib')

# Perform preprocessing on categorical features
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
X_encoded = encoder.fit_transform(X[cat_features])

# Save the encoder
dump(encoder, 'encoder.joblib')

# Combine preprocessed numerical and categorical features
X_processed = np.concatenate((X_encoded, X[num_features]), axis=1)

# Create an instance of LabelEncoder
label_encoder = LabelEncoder()

# Encode the categorical labels into integer values
y_encoded = label_encoder.fit_transform(y)

# Save the label encoder
dump(label_encoder, 'label_encoder.joblib')

<ipython-input-9-4039d8068e11>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[num_features] = scaler.fit_transform(X[num_features])


['label_encoder.joblib']

#Split Data

In [ ]:
# Split the subset into training and validation sets
X_train, X_temp, y_train, y_temp = train_test_split(X_processed, y_encoded, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)